In [1]:
import os
import sys
sys.path.append(os.path.abspath("src"))

In [2]:
%pwd 

'c:\\Users\\91810\\Documents\\mlops\\mlops_first_project\\research'

change directory to parent dir

In [3]:
os.chdir("C:/Users/91810/Documents/mlops/mlops_first_project")
%pwd

'C:\\Users\\91810\\Documents\\mlops\\mlops_first_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestconfig:
    root_dir: Path
    source_URL: str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from src.datascienceproject.constants import *
from src.datascienceproject.util.common import read_yaml,create_directories
from pathlib import Path

Reads a YAML file and returns its contents as an object with dot-access
Run successfully
more yaml generic functions create
Creates one or more directories. Automatically skip existing ones
Saves a dictionary as a .json file. Metadata , Predictions , Evaluations
Loads a .json file and returns a ConfigBox (dot-accessible dict).
Saves Any Python object (model, list, dictionary, etc.) to a file in binary format.
Loads the binary file and returns the original Python object.


In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schemas_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schemas_filepath))

        create_directories([self.config.artifacts_root],verbose = True)

    def get_data_ingestion_config(self)-> DataIngestconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir],verbose = True)

        data_ingestion_config = DataIngestconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

components data ingestion

In [7]:
import urllib.request as request
from src.datascienceproject import logger
import zipfile

In [23]:
import os
from urllib import request
from src.datascienceproject import logger  # adjust as per your logger module

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        url = str(self.config.source_URL).replace("\\", "/").replace("https:/", "https://")
        local_file = str(self.config.local_data_file).replace("\\", "/")

        logger.info(f"About to download from URL: '{url}'")
        print("URL before download:", url)
        print("Local file path:", local_file)

        if not os.path.exists(local_file):
            filename, headers = request.urlretrieve(
                url=url,
                filename=local_file
            )
            logger.info(f"{filename} downloaded with info: \n{headers}")
        else:
            logger.info(f"File already exists: {local_file}")



    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()

except Exception as e:
    raise e

[2025-06-28 18:05:59,347:INFO:common:yaml file :config\config.yaml loaded successfully]
[2025-06-28 18:05:59,351:INFO:common:yaml file :params.yaml loaded successfully]
[2025-06-28 18:05:59,355:INFO:common:yaml file :schema.yaml loaded successfully]
[2025-06-28 18:05:59,359:INFO:common:created directory at:artifacts]
[2025-06-28 18:05:59,362:INFO:common:created directory at:artifacts\data_ingestion]
[2025-06-28 18:05:59,363:INFO:3172246662:About to download from URL: 'https://raw.githubusercontent.com/Yash49hdfmj/mlops_first_project/main/data.zip']
URL before download: https://raw.githubusercontent.com/Yash49hdfmj/mlops_first_project/main/data.zip
Local file path: artifacts/data_ingestion/data.zip
[2025-06-28 18:06:00,417:INFO:3172246662:artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae5

In [ ]:
import inspect
print(create_directories)  # check what object it is
print(inspect.signature(create_directories))  # see what Python thinks the function takes


<function create_directories at 0x000001937F918DC0>
(path_to_directories: list, verbose: bool = True)


In [ ]:
from importlib import reload
from src.datascienceproject import util
reload(util)
from src.datascienceproject.util.common import create_directories
